In [4]:
# import packages

import numpy as np
import pandas as pd
import torch
import matplotlib as plt

In [7]:
# loading the data

df = pd.read_csv('../data/spotify_dataset.csv')


In [8]:
# Initial overview

print(df.shape)

(114000, 21)
